In [1]:
import os
import pandas as pd
import numpy as np
import joblib
import sys


feature_path = os.path.join(os.getcwd(), '..', 'src', 'features')
visualize_path = os.path.join(os.getcwd(), '..', 'src', 'visualization')
sys.path.append(feature_path)
sys.path.append(visualize_path)


import build_features # type: ignore
import visualize # type: ignore


data_path = os.path.join('..', 'data', 'interim', '01_H_Glycine_processed.pkl')

preprocessor = joblib.load('preprocessor')
df = pd.read_pickle(data_path)
df.head()

,SMILES,ACTIVITY,ACTIVITY_SCORE,ACTIVATION_AT_6.8uM
PUBCHEM_RESULT_TAG,,,,
1,C1=CC=C(C(=C1)NC(=O)C2=CC=C(C=C2)NS(=O)(=O)C3=...,1,100.0,89.39
2,C1=CC=C(C(=C1)N)NC(=O)C2=CC=C(C=C2)N,1,95.0,85.43
3,C1=CC=C(C=C1)C(=O)C2=CC=C(C=C2)C(=O)NC3=C(C=CC...,1,94.0,84.24
4,C1=CC=C(C=C1)NS(=O)(=O)C2=CC=C(C=C2)C(=O)NC3=C...,1,90.0,80.91
5,C1CC1C(=O)NC2=CC=C(C=C2)C(=O)NC3=CC=CC=C3O,1,77.0,68.96


In [2]:
train_smiles = pd.read_pickle(os.path.join('..', 'data', 'interim', '01_H_Contortus_processed.pkl')).SMILES

In [3]:
df = df[~df['SMILES'].isin(train_smiles)]

In [4]:
smiles_list = df['SMILES'].to_list()
mordred_df = build_features.mordred_descriptors(smiles_list)
mordred_df.index = df.index
mordred_df['ACTIVITY'] = df['ACTIVITY']


empty_cols = mordred_df.columns[mordred_df.isna().sum() == 3416]
mordred_df.drop(columns=empty_cols, inplace=True)
mordred_df.head()

d:\Users\Gabriel Anyaele\Desktop\daf12-antihelminthic-drugdiscovery\venv\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


d:\Users\Gabriel Anyaele\Desktop\daf12-antihelminthic-drugdiscovery\notebooks\..\src\features\build_features.py:77: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df_mordred.applymap(float_converter)


,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,ACTIVITY
PUBCHEM_RESULT_TAG,,,,,,,,,,,,,,,,,,,,,
6,NaN,NaN,0,0,29.235755,2.363527,4.598328,29.235755,1.328898,4.015802,...,70.787850,296.127326,7.792824,1290,26,110.0,125.0,5.555556,4.972222,1
8,NaN,NaN,0,0,27.687761,2.370851,4.566530,27.687761,1.318465,3.973837,...,69.658190,282.100442,8.060013,1104,25,106.0,121.0,5.305556,4.722222,1
10,NaN,NaN,0,0,20.903705,2.300925,4.601850,20.903705,1.306482,3.686078,...,47.894655,212.094963,7.574820,460,21,78.0,88.0,4.944444,3.638889,1
11,NaN,NaN,0,0,26.705830,2.366309,4.602455,26.705830,1.335291,3.930032,...,68.498800,272.116092,7.558780,923,24,102.0,117.0,5.055556,4.472222,1
12,NaN,NaN,0,1,29.167842,2.370361,4.556479,29.167842,1.325811,4.015803,...,70.800745,321.114712,7.832066,1301,26,110.0,125.0,5.555556,4.972222,1


## loading models

In [8]:
import pickle

with open(os.path.join('..', 'models', 'rfc_model.pkl'), 'rb') as file: 
    rfc_model = pickle.load(file) 


with open(os.path.join('..', 'models', 'svc_model.pkl'), 'rb') as file: 
    svc_model = pickle.load(file) 
    
    
with open(os.path.join('..', 'models', 'knn_model.pkl'), 'rb') as file: 
    knn_model = pickle.load(file) 
    
    
with open(os.path.join('..', 'models', 'lr_model.pkl'), 'rb') as file: 
    lr_model = pickle.load(file) 


In [6]:
from sklearn.metrics import roc_auc_score

validation_data = mordred_df
validation_label = mordred_df.ACTIVITY

## Validation

In [9]:
rfc_pred = rfc_model.predict(validation_data)

print('roc_auc_score: ', round(roc_auc_score(validation_label, rfc_model.predict_proba(validation_data)[:, 1]), 3))
visualize.score(validation_label, rfc_pred)

roc_auc_score:  0.678


{'accuracy_score': 0.665629168519342,
 'precision_score': 0.8240343347639485,
 'recall_score': 0.6948130277442702,
 'f1_score': 0.7539267015706806}

In [10]:
svc_pred = svc_model.predict(validation_data)

print('roc_auc_score: ', round(roc_auc_score(validation_label, svc_model.predict_proba(validation_data)[:, 1]), 3))
visualize.score(validation_label, svc_pred)

roc_auc_score:  0.685


{'accuracy_score': 0.6860827034237439,
 'precision_score': 0.8160690571049137,
 'recall_score': 0.741254523522316,
 'f1_score': 0.7768647281921618}

In [11]:
lr_pred = lr_model.predict(validation_data)

print('roc_auc_score: ', round(roc_auc_score(validation_label, lr_model.predict_proba(validation_data)[:, 1]), 3))
visualize.score(validation_label, lr_pred)

roc_auc_score:  0.631


{'accuracy_score': 0.609159626500667,
 'precision_score': 0.8003084040092521,
 'recall_score': 0.6260554885404102,
 'f1_score': 0.7025380710659899}

In [12]:
knn_pred = knn_model.predict(validation_data)

print('roc_auc_score: ', round(roc_auc_score(validation_label, knn_model.predict_proba(validation_data)[:, 1]), 3))
visualize.score(validation_label, knn_pred)

roc_auc_score:  0.605


{'accuracy_score': 0.6705202312138728,
 'precision_score': 0.7740585774058577,
 'recall_score': 0.7810615199034981,
 'f1_score': 0.7775442809966977}